Imports needen only in Colab

In [ ]:
# !pip install -q tensorflow-gpu==2.0.0-beta0

# import os
# from google.colab import drive
# drive.mount('/content/drive/')
# try:
#     os.chdir("drive/My Drive/snr/project")
# except FileNotFoundError:
#     pass

All other imports

In [ ]:
import os

import tensorflow as tf
from tensorflow import keras

AUTOTUNE = tf.data.experimental.AUTOTUNE

import numpy as np

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from definitions import *
from preprocessing import load_datasets, time_footprint

# Load datasets

In [ ]:
train_ds, test_ds, val_ds = load_datasets()

# Create model

In [ ]:
ALPHA = 1.0  # controls the width of the network
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape = IMG_SHAPE,
                                               include_top = False, 
                                               alpha       = ALPHA,
                                               weights     = 'imagenet')

**MobileNetV2** model's last 3 layers are: **Conv2D**, **BatchNormalization**, **ReLU**. So to make last convolution layer trainable we need to set all layers, expect last 3, as non-trainable.

In [ ]:
base_model.trainable = True
for layer in base_model.layers[:-3]:
    layer.trainable = False

In [ ]:
model = tf.keras.Sequential([
    base_model,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(NUMBER_OF_LABELS, activation='softmax')
])

In [ ]:
base_learning_rate = 2.5e-5
model.compile(optimizer = tf.keras.optimizers.Adam(lr=base_learning_rate), 
              loss      = 'sparse_categorical_crossentropy', 
              metrics   = ['accuracy'])

In [ ]:
model.summary()

# Train model

In [ ]:
epochs          = 150
steps_per_epoch = TRAIN_FILES//BATCH_SIZE
val_steps       =   VAL_FILES//BATCH_SIZE

time_fp = time_footprint()
log_dir = os.path.join(LOG_DIR, 'task_2', time_fp)
os.makedirs(log_dir)

checkpoint_path = log_dir + "/task_2_" + time_fp + ".ckpt"


callbacks = [
    keras.callbacks.TensorBoard(log_dir=log_dir),
    tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                       save_weights_only=True,
                                       verbose=1,
                                       save_best_only=True,
                                       monitor='val_loss')
]


history = model.fit(train_ds,
                    epochs           = epochs,
                    steps_per_epoch  = steps_per_epoch,
                    validation_data  = val_ds, 
                    validation_steps = val_steps,
                    callbacks        = callbacks)


val_acc = history.history['val_accuracy'][0]

model_name = "task_2_" + time_fp + '_acc_' + f'{val_acc:.4f}' + '.h5'
model.save(MODEL_DIR + '/' + model_name)